In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rpy2.robjects import FloatVector, pandas2ri
from rpy2.robjects.packages import importr
from scipy.interpolate import interp1d
from scipy.stats import norm
from scipy.optimize import root_scalar
import os
import glob


# Define the root directory containing the subfolders
root_dir = os.path.expanduser("~/fdp-estimation/results/results_subsampling")

# Find all subfolders in the root directory
subfolders = [f.path for f in os.scandir(root_dir) if f.is_dir()]

# Placeholder for storing overall results
overall_results = []


# --- Golden Curve Computation ---
# Define parameters
mu = 1  # sense = 1 implies mu = 1
m=5
n=10
p = m / n

# Activate pandas2ri for DataFrame conversion between Python and R
pandas2ri.activate()

# Load the necessary R packages
fdrtool = importr('fdrtool')


In [2]:
# Define parameters
# # Define Gaussian_curve function
def Gaussian_curve(alpha):
    return norm.cdf(norm.ppf(1 - alpha) - mu)

# Define f_p_true function
def f_p_true(alpha):
    return p * Gaussian_curve(alpha) + (1 - p) * (1 - alpha)

# Fixed point calculation
def fixed_point(x):
    return Gaussian_curve(x) - x

fix_point = root_scalar(fixed_point, bracket=[0, 1]).root

# Define C_p_true function
def C_p_true(x, fix_x=fix_point):
    if x <= fix_x:
        return f_p_true(x)
    elif fix_x < x <= f_p_true_inv(fix_x):
        return fix_x + f_p_true(fix_x) - x
    else:
        return f_p_true_inv(x)

In [3]:

# Iterate through each subfolder
for subfolder in subfolders:
    print(f"Processing subfolder: {subfolder}")

    # Find all CSV files in the current subfolder
    csv_files = glob.glob(os.path.join(subfolder, "*.csv"))

    # Placeholder for storing subfolder-specific results
    subfolder_results = []

    for file in csv_files:
        # Read the CSV file
        data = pd.read_csv(file)

        # Step 1: Remove duplicate alpha values
        output_df_unique = data.drop_duplicates(subset="alpha")

        # Step 2: Create interpolation functions for f_p_values and f_p_values_inv
        f_p_values = interp1d(output_df_unique["alpha"], output_df_unique["beta"], fill_value="extrapolate")
        f_p_values_inv = interp1d(output_df_unique["beta"], output_df_unique["alpha"], fill_value="extrapolate")

        # Step 3: Define min_func to compute the minimum of f_p_values and f_p_values_inv
        def min_func(x):
            return np.minimum(f_p_values(x), f_p_values_inv(x))

        # Step 4: Generate alpha values for computation
        alpha_values = output_df_unique["alpha"]

        # Step 5: Compute min_func values
        min_func_vals = min_func(alpha_values)

        # Step 6: Convert data to R vectors for gcmlcm
        r_alpha_values = FloatVector(alpha_values)
        r_min_func_vals = FloatVector(min_func_vals)

        # Step 7: Call the gcmlcm function from the fdrtool package
        gcmlcm_result = fdrtool.gcmlcm(r_alpha_values, r_min_func_vals)

        # Step 8: Extract the result from the gcmlcm output
        gcmlcm_x = np.array(gcmlcm_result.rx2('x.knots'))
        gcmlcm_y = np.array(gcmlcm_result.rx2('y.knots'))

        # Step 9: Compute f_p_values and f_p_values_inv for plotting
        f_p_values_vals = f_p_values(alpha_values)
        f_p_values_inv_vals = f_p_values_inv(alpha_values)

        # Step 10: Create a DataFrame for plotting
        plot_data = pd.DataFrame({
            "alpha": alpha_values,
            "beta": min_func_vals
        })

        # Ensure the CSV contains 'alpha' and 'beta' columns
        if 'alpha' not in plot_data.columns or 'beta' not in plot_data.columns:
            print(f"Skipping {file}: Missing required columns 'alpha' and 'beta'.")
            continue

        # Filter the data to only include rows where 0 <= alpha <= 1
        filtered_data = plot_data[(plot_data['alpha'] >= 0) & (plot_data['alpha'] <= 1)]

        # Skip if no valid rows remain
        if filtered_data.empty:
            print(f"Skipping {file}: No valid alpha values between 0 and 1.")
            continue

        # Compute the maximum error for the current file
        alpha_values = filtered_data['alpha'].values
        beta_values = filtered_data['beta'].values

        # --- Golden Curve Computation ---
        gauss_values = Gaussian_curve(alpha_values)
        # Compute C_p_values_true only for the valid alpha_values
        f_p_values_true = f_p_true(alpha_values)
        f_p_true_inv = interp1d(f_p_values_true, alpha_values, bounds_error=False, fill_value="extrapolate")
        C_p_values_true = np.array([C_p_true(alpha, fix_x=fix_point) for alpha in alpha_values])

        # Check lengths to ensure they match
        if len(alpha_values) != len(C_p_values_true):
            raise ValueError("alpha_values and C_p_values_true lengths do not match!")
        # Create DataFrame for golden curve
        C_p_data_true = pd.DataFrame({
            "alpha": alpha_values,
            "beta": C_p_values_true
        })
        f_values = C_p_data_true['beta']
        max_error = np.max(np.abs(f_values - beta_values))

        # Store the result
        result = {
            "file": file,
            "max_error": max_error
        }
        subfolder_results.append(result)

    # Convert subfolder results to a DataFrame
    subfolder_df = pd.DataFrame(subfolder_results)


    # Save the subfolder-specific results to a new CSV
    output_file = os.path.join(subfolder, "max_errors.csv")
    subfolder_df.to_csv(output_file, index=False)
    print(f"Saved maximum errors for subfolder: {subfolder}")

    # Append to overall results
    overall_results.extend(subfolder_results)

# Convert overall results to a DataFrame
overall_df = pd.DataFrame(overall_results)

# Save the overall results
overall_output_file = os.path.join(root_dir, "overall_max_errors.csv")
overall_df.to_csv(overall_output_file, index=False)
print(f"Saved overall maximum errors to: {overall_output_file}")

# Extract alpha and beta for plotting
#alpha = output_df["alpha"]
#beta = output_df["beta"]

Processing subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N1000
Saved maximum errors for subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N1000
Processing subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N1000000
Saved maximum errors for subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N1000000
Processing subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N10000
Saved maximum errors for subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N10000
Processing subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N100000
Saved maximum errors for subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N100000
Processing subfolder: /home/martin/fdp-estimation/results/results_subsampling/results_subsampling_N1

In [4]:
def compute_mse_from_csv(file_path):
    data = pd.read_csv(file_path)
    errors = data["max_error"].values
    
    # Compute the Mean Squared Error
    mse = np.mean(errors)**2+np.var(errors)
    return mse
mse_100=compute_mse_from_csv("~/fdp-estimation/results/results_subsampling/results_subsampling_N100/max_errors.csv")
mse_1000=compute_mse_from_csv("~/fdp-estimation/results/results_subsampling/results_subsampling_N1000/max_errors.csv")
mse_10000=compute_mse_from_csv("~/fdp-estimation/results/results_subsampling/results_subsampling_N10000/max_errors.csv")
mse_100000=compute_mse_from_csv("~/fdp-estimation/results/results_subsampling/results_subsampling_N100000/max_errors.csv")
mse_1000000=compute_mse_from_csv("~/fdp-estimation/results/results_subsampling/results_subsampling_N1000000/max_errors.csv")
mse_values = {
    "N": [100, 1000, 10000, 100000, 1000000],
    "Mean Squared Error": [
        # Replace these with the actual MSE values if known
        mse_100, 
        mse_1000, 
        mse_10000, 
        mse_100000, 
        mse_1000000
    ]
}
mse_df = pd.DataFrame(mse_values)
mse_df.to_csv("mse_values.csv", index=False)